In [1]:
import pandas as pd
import numpy as np

# Load the dataset
df_sales = pd.read_csv('Sales.csv')

# 1. Handle missing values

# Fill missing 'Customer Name' with "Unknown"
if 'Customer Name' in df_sales.columns:
    df_sales['Customer Name'] = df_sales['Customer Name'].fillna('Unknown')

# Fill missing 'Quantity' with median
if 'Quantity' in df_sales.columns:
    df_sales['Quantity'] = df_sales['Quantity'].fillna(df_sales['Quantity'].median())

# Calculate missing 'Unit Price' where 'Total Revenue' and 'Quantity' are present and valid
if {'Unit Price', 'Total Revenue', 'Quantity'}.issubset(df_sales.columns):
    mask = (df_sales['Unit Price'].isna()) & (df_sales['Total Revenue'].notna()) & (df_sales['Quantity'] > 0)
    df_sales.loc[mask, 'Unit Price'] = df_sales.loc[mask, 'Total Revenue'] / df_sales.loc[mask, 'Quantity']

    # Fill remaining missing 'Unit Price' with median per Product
    df_sales['Unit Price'] = df_sales.groupby('Product')['Unit Price'].transform(lambda x: x.fillna(x.median()))

# Calculate missing 'Total Revenue' where possible
if {'Total Revenue', 'Unit Price', 'Quantity'}.issubset(df_sales.columns):
    mask = (df_sales['Total Revenue'].isna()) & (df_sales['Unit Price'].notna()) & (df_sales['Quantity'].notna())
    df_sales.loc[mask, 'Total Revenue'] = df_sales.loc[mask, 'Unit Price'] * df_sales.loc[mask, 'Quantity']

# 2. Fix inconsistent date formats
if 'Order Date' in df_sales.columns:
    df_sales['Order Date'] = df_sales['Order Date'].astype(str).str.replace("'", "", regex=False)
    df_sales['Order Date'] = pd.to_datetime(df_sales['Order Date'], errors='coerce')

# 3. Remove duplicate rows
df_sales = df_sales.drop_duplicates()

# 4. Fix wrong data
# Convert negative 'Quantity' and 'Total Revenue' to positive values
if 'Quantity' in df_sales.columns:
    df_sales['Quantity'] = df_sales['Quantity'].apply(lambda x: abs(x) if x < 0 else x)

if 'Total Revenue' in df_sales.columns:
    df_sales['Total Revenue'] = df_sales['Total Revenue'].apply(lambda x: abs(x) if x < 0 else x)

# 5. No unnecessary columns found — skipping column drop

# Save cleaned dataset
df_sales.to_csv('Sales_cleaned.csv', index=False)

# Confirmation message
print("✅ Sales data cleaned and saved to 'Sales_cleaned.csv'")


✅ Sales data cleaned and saved to 'Sales_cleaned.csv'
